In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
# %load_ext cudf.pandas
# import pandas as pd
# print(pd)


In [ ]:
# Optimizations
# GDAL optimizations
import multiprocessing as mp
import os

cpu_count: int = mp.cpu_count()
num_cores: int = cpu_count - 2
os.environ["GDAL_NUM_THREADS"] = f"{num_cores}"
os.environ["GDAL_CACHEMAX"] = "1024"


## Libraries

In [ ]:
# Imports
import os
from pathlib import Path

import numpy as np
import pandas as pd
import riskmapjnr as rmj


In [ ]:
# Add root to path
import sys

sys.path.append("..")
from component.script.utilities.file_filter import (
    list_files_by_extension,
    filter_files_by_keywords,
)


## Set user parameters

In [ ]:
project_name = "test"


In [ ]:
forest_source = "gfc"  ##gfc, tmf
tree_cover_threshold = 10
years = [2015, 2020, 2024]
string_years = [str(num) for num in years]


In [ ]:
win_size_list = [5, 11, 21]
block_rows = 256


## Connect folders

In [ ]:
root_folder: Path = Path.cwd().parent
downloads_folder: Path = root_folder / "data"
downloads_folder.mkdir(parents=True, exist_ok=True)


In [ ]:
project_folder = downloads_folder / project_name
project_folder.mkdir(parents=True, exist_ok=True)
processed_data_folder = project_folder / "data"
processed_data_folder.mkdir(parents=True, exist_ok=True)
plots_folder = project_folder / "plots"
plots_folder.mkdir(parents=True, exist_ok=True)
rmj_mw = project_folder / "rmj_mw"
rmj_mw.mkdir(parents=True, exist_ok=True)


## Forest Files

In [ ]:
# List all raster files in the processed data folder
input_raster_files = list_files_by_extension(processed_data_folder, [".tiff", ".tif"])


In [ ]:
forest_change_file = filter_files_by_keywords(
    input_raster_files,
    ["forest", "loss", forest_source] + string_years,
    False,
    ["distance", "edge"],
)[0]


In [ ]:
forest_yearly_files = filter_files_by_keywords(
    input_raster_files, ["forest", forest_source], False, ["loss", "edge"]
)


In [ ]:
forest_edge_files = filter_files_by_keywords(
    input_raster_files, ["forest", forest_source, "edge"], False
)


In [ ]:
raster_subj_file = filter_files_by_keywords(input_raster_files, ["subj"])[0]


## Periods dictionaries

In [ ]:
calibration_dict = {
    "period": "calibration",
    "train_period": "calibration",
    "initial_year": years[0],
    "final_year": years[1],
    "defor_value": 1,
    "time_interval": years[1] - years[0],
    "forest": filter_files_by_keywords(forest_yearly_files, [str(years[0])])[0],
    "forest_edge": filter_files_by_keywords(forest_edge_files, [str(years[0])])[0],
}
validation_dict = {
    "period": "validation",
    "train_period": "calibration",
    "initial_year": years[1],
    "final_year": years[2],
    "defor_value": 1,
    "time_interval": years[2] - years[1],
    "forest": filter_files_by_keywords(forest_yearly_files, [str(years[1])])[0],
    "forest_edge": filter_files_by_keywords(forest_edge_files, [str(years[1])])[0],
}
historical_dict = {
    "period": "historical",
    "train_period": "historical",
    "initial_year": years[0],
    "final_year": years[2],
    "defor_value": [1, 2],
    "time_interval": years[2] - years[0],
    "forest": filter_files_by_keywords(forest_yearly_files, [str(years[0])])[0],
    "forest_edge": filter_files_by_keywords(forest_edge_files, [str(years[0])])[0],
}
forecast_dict = {
    "period": "forecast",
    "train_period": "historical",
    "initial_year": years[0],
    "final_year": years[2],
    "defor_value": [1, 2],
    "time_interval": years[2] - years[0],
    "forest": filter_files_by_keywords(forest_yearly_files, [str(years[2])])[0],
    "forest_edge": filter_files_by_keywords(forest_edge_files, [str(years[2])])[0],
}


## 1 Calculate distance to forest edge

In [ ]:
deforestation_thresh = 99.5
max_dist = 5000


In [ ]:
def calculate_period_dist_edge_threshold(
    forest_change_file,
    period_dictionary,
    deforestation_thresh,
    max_dist,
    model_folder,
    plots_folder,
):
    period_output_folder = model_folder / period_dictionary["period"]
    period_output_folder.mkdir(parents=True, exist_ok=True)

    dist_thresh = rmj.dist_edge_threshold(
        fcc_file=forest_change_file,
        defor_values=period_dictionary["defor_value"],
        defor_threshold=deforestation_thresh,
        dist_file=period_dictionary["forest_edge"],
        dist_bins=np.arange(0, max_dist, step=30),
        tab_file_dist=period_output_folder / "tab_dist.csv",
        # fig_file_dist= period_output_folder /"perc_dist.png",
        fig_file_dist=plots_folder / f"perc_dist_{period_dictionary['period']}.png",
        blk_rows=block_rows,
        dist_file_available=True,
        check_fcc=False,
        verbose=False,
    )
    # Save result
    dist_edge_data = pd.DataFrame(dist_thresh, index=[0])
    dist_edge_data.to_csv(
        period_output_folder / "dist_edge_threshold.csv",
        sep=",",
        header=True,
        index=False,
        index_label=False,
    )


In [ ]:
calculate_period_dist_edge_threshold(
    forest_change_file,
    calibration_dict,
    deforestation_thresh,
    max_dist,
    rmj_mw,
    plots_folder,
)


In [ ]:
calculate_period_dist_edge_threshold(
    forest_change_file,
    historical_dict,
    deforestation_thresh,
    max_dist,
    rmj_mw,
    plots_folder,
)


## 2 Compute local_defor_rate

In [ ]:
def calculate_local_defor_rate(
    forest_change_file, win_size_list, period_dictionary, model_folder
):
    period_output_folder = model_folder / period_dictionary["period"]
    period_output_folder.mkdir(parents=True, exist_ok=True)
    trained_period_output_folder = model_folder / period_dictionary["train_period"]
    trained_period_output_folder.mkdir(parents=True, exist_ok=True)

    for win_size in win_size_list:
        ldefrate_file = period_output_folder / f"ldefrate_mw_{win_size}.tif"
        rmj.local_defor_rate(
            fcc_file=forest_change_file,
            defor_values=period_dictionary["defor_value"],
            ldefrate_file=ldefrate_file,
            win_size=win_size,
            time_interval=period_dictionary["time_interval"],
            rescale_min_val=2,
            rescale_max_val=65535,
            blk_rows=block_rows,
            verbose=False,
        )


In [ ]:
calculate_local_defor_rate(forest_change_file, win_size_list, calibration_dict, rmj_mw)


In [ ]:
calculate_local_defor_rate(forest_change_file, win_size_list, historical_dict, rmj_mw)


## 3 Compute prediction

In [ ]:
def get_dist_thresh(ifile):
    """Get distance to forest edge threshold."""
    dist_thresh_data = pd.read_csv(ifile)
    dist_thresh = dist_thresh_data.loc[0, "dist_thresh"]
    return dist_thresh


def calculate_period_mw_prediction(period_dictionary, win_size_list, model_folder):
    period_output_folder = model_folder / period_dictionary["period"]
    period_output_folder.mkdir(parents=True, exist_ok=True)

    trained_period_output_folder = model_folder / period_dictionary["train_period"]
    period_dist_edge_file = trained_period_output_folder / "dist_edge_threshold.csv"

    for win_size in win_size_list:
        ldefrate_file = trained_period_output_folder / f"ldefrate_mw_{win_size}.tif"
        output_file = (
            period_output_folder
            / f"prob_mw_{win_size}_{period_dictionary['period']}.tif"
        )
        rmj.set_defor_cat_zero(
            ldefrate_file=ldefrate_file,
            dist_file=period_dictionary["forest_edge"],
            dist_thresh=get_dist_thresh(period_dist_edge_file),
            ldefrate_with_zero_file=output_file,
            blk_rows=block_rows,
            verbose=False,
        )


In [ ]:
calculate_period_mw_prediction(calibration_dict, win_size_list, rmj_mw)


In [ ]:
calculate_period_mw_prediction(historical_dict, win_size_list, rmj_mw)


In [ ]:
calculate_period_mw_prediction(validation_dict, win_size_list, rmj_mw)


In [ ]:
calculate_period_mw_prediction(forecast_dict, win_size_list, rmj_mw)


## 4 Compute deforestation rate per class

In [ ]:
def calculate_period_mw_def_rate(
    forest_change_file, period_dictionary, win_size_list, model_folder
):
    period_output_folder = model_folder / period_dictionary["period"]
    if not os.path.exists(period_output_folder):
        os.makedirs(period_output_folder)
    trained_period_output_folder = model_folder / period_dictionary["train_period"]
    period_dist_edge_file = trained_period_output_folder / "dist_edge_threshold.csv"

    for win_size in win_size_list:
        ldefrate_file = trained_period_output_folder / f"ldefrate_mw_{win_size}.tif"
        riskmap_file = (
            period_output_folder
            / f"prob_mw_{win_size}_{period_dictionary['period']}.tif"
        )
        output_file = (
            period_output_folder
            / f"defrate_cat_mw_{win_size}_{period_dictionary['period']}.csv"
        )

        rmj.defrate_per_cat(
            fcc_file=forest_change_file,
            riskmap_file=riskmap_file,
            time_interval=period_dictionary["time_interval"],
            period=period_dictionary["period"],
            tab_file_defrate=output_file,
            blk_rows=block_rows,
            verbose=False,
        )


In [ ]:
calculate_period_mw_def_rate(
    forest_change_file, calibration_dict, win_size_list, rmj_mw
)


In [ ]:
calculate_period_mw_def_rate(forest_change_file, historical_dict, win_size_list, rmj_mw)


In [ ]:
calculate_period_mw_def_rate(forest_change_file, validation_dict, win_size_list, rmj_mw)


In [ ]:
calculate_period_mw_def_rate(forest_change_file, forecast_dict, win_size_list, rmj_mw)


## 5 Plot Risk map of deforestation

In [ ]:
input_vector_files = list_files_by_extension(processed_data_folder, [".shp"])
aoi_vector = filter_files_by_keywords(input_vector_files, ["aoi"])[0]


In [ ]:
def calculate_period_plot_risk_map(
    period_dictionary, aoi_vector, model_folder, plots_folder
):
    period_output_folder = model_folder / period_dictionary["period"]
    raster_files = list_files_by_extension(period_output_folder, [".tiff", ".tif"])
    prob_mw_files = filter_files_by_keywords(
        raster_files, ["prob", "mw"], False, ["ldefrate"]
    )
    for prob_mw_file in prob_mw_files:
        ifile = prob_mw_file
        base_name = os.path.splitext(os.path.basename(ifile))[0]
        png_filename = os.path.join(plots_folder, f"{base_name}.png")
        riskmap_fig = rmj.plot.riskmap(
            input_risk_map=ifile,
            maxpixels=1e8,
            output_file=png_filename,
            borders=aoi_vector,
            legend=False,
            figsize=(6, 5),
            dpi=100,
            linewidth=0.3,
        )


In [ ]:
calculate_period_plot_risk_map(calibration_dict, aoi_vector, rmj_mw, plots_folder)


In [ ]:
calculate_period_plot_risk_map(historical_dict, aoi_vector, rmj_mw, plots_folder)


In [ ]:
calculate_period_plot_risk_map(validation_dict, aoi_vector, rmj_mw, plots_folder)


In [ ]:
calculate_period_plot_risk_map(forecast_dict, aoi_vector, rmj_mw, plots_folder)
